In [ ]:
!pip install numpy opencv-python matplotlib torch torchvision scipy tqdm ultralytics

In [ ]:
!pip install catboost
!pip install scenedetect
!pip install streamlit -q
!pip install pyngrok
!pip -q install openai

In [ ]:
from pyngrok import ngrok

# ngrok API 키
ngrok.set_auth_token('')

In [ ]:
%%writefile app.py

import os
import sys
import cv2
import base64
import subprocess
import numpy as np
import streamlit as st
from openai import OpenAI
from pyngrok import ngrok
from PIL import Image, ImageOps

# 모든 API 키 -----------------------------------------
# OpenAI API 키를 하드코딩하여 설정
api_key = ""  # 여기에 API 키를 입력하세요

# API 키를 환경 변수에 저장
os.environ['OPENAI_API_KEY'] = api_key

# -----------------------------------------
# 모델 실행 함수 -----------------------------------------
def process_video(input_video_path, output_video_path, ball_model, court_model, bounce_model):
    """
    동영상을 처리하는 함수.

    매개변수:
        input_video_path (str): 입력 동영상 파일 경로
        output_video_path (str): 처리된 동영상 출력 경로
        ball_model (str): 공 추적 모델 경로
        court_model (str): 코트 검출 모델 경로
        bounce_model (str): 바운스 검출 모델 경로
    """
    command = [
        "python", "/content/TennisProject/main.py",
        "--path_ball_track_model", "/content/ball_model.pt",
        "--path_court_model", "/content/court_detection.pt",
        "--path_bounce_model", "/content/bounce_detection.cbm",
        "--path_input_video", input_video_path,
        "--path_output_video", "/content/tennis_detection.mp4"
    ]

    # subprocess.run으로 외부 명령어 실행
    result = subprocess.run(
        command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    if result.returncode != 0:
        # 실행 실패 시 에러 메시지 반환
        return False, result.stderr
    else:
        # 실행 성공 시 성공 메시지 반환
        return True, result.stdout
# -----------------------------------------
# 프레임 별 이미지 자르는 함수 -----------------------------------------
def extract_frames_from_video(video_path, output_folder='/content/image', frame_rate=1):
    """
    동영상에서 일정 간격으로 프레임을 추출하여 저장하는 함수

    매개변수:
        video_path (str): 동영상 파일 경로
        output_folder (str): 추출된 프레임을 저장할 폴더 경로 (기본값: '/content/image')
        frame_rate (int): 초당 추출할 프레임 수 (기본값: 1초당 1프레임)
    """
    # 동영상 파일 로드
    cap = cv2.VideoCapture(video_path)

    # 동영상 파일이 열렸는지 확인
    if not cap.isOpened():
        print("오류: 동영상을 열 수 없습니다.")
        return

    # 저장할 폴더가 없으면 생성
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"폴더 생성됨: {output_folder}")

    # 동영상의 FPS(초당 프레임 수) 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS)  # FPS: Frames Per Second
    frame_interval = int(fps / frame_rate)  # 추출할 프레임 간격 계산

    frame_count = 0  # 현재 프레임 번호
    saved_frame_count = 0  # 저장된 프레임 수

    while True:
        # 동영상에서 다음 프레임 읽기
        ret, frame = cap.read()

        # 더 이상 프레임이 없으면 반복 종료
        if not ret:
            break

        # 지정된 간격에 해당하는 프레임만 저장
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f'image{saved_frame_count:04d}.jpg')  # 파일 이름 생성
            cv2.imwrite(frame_filename, frame)  # 프레임 저장
            print(f"저장됨: {frame_filename}")  # 저장 완료 메시지 출력
            saved_frame_count += 1  # 저장된 프레임 수 증가

        # 현재 프레임 번호 증가
        frame_count += 1

    # 동영상 파일 닫기
    cap.release()
    # 모든 OpenCV 창 닫기
    cv2.destroyAllWindows()
# -----------------------------------------
# openai API 함수 -----------------------------------------

# OpenAI 클라이언트 초기화
client = OpenAI()

# 이미지를 base64로 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# OpenAI API 호출 함수
def interpret_images_with_openai(image_paths):
    # 각 이미지 경로를 base64로 변환
    base64_images = [
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encode_image(path)}"}}
        for path in image_paths
    ]

    # OpenAI API 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": [{"type": "text", "text": "이미지가 어떤 흐름으로 흘러가는지 해석해줘"}] + base64_images}
        ],
    )

    # OpenAI의 응답에서 텍스트 추출
    return response.choices[0].message.content
# -----------------------------------------

st.title("테니스 영상 해설")
file = st.file_uploader("영상을 업로드하세요.", type=['mp4', 'avi', 'mov'])

if file is None:
    st.text("동영상을 먼저 업로드해주세요.")
else:
    # 동영상 저장 경로
    input_video_path = "/content/tennis.mp4"
    with open(input_video_path, "wb") as f:
        f.write(file.read())
    st.success("동영상이 성공적으로 업로드되었습니다!")

    # 명령어 실행
    st.text("동영상을 처리 중입니다...")
    input_video_path = "/content/tennis.mp4"
    output_video_path = "/content/tennis_detection.mp4"
    ball_model_path = "/content/ball_model.pt"
    court_model_path = "/content/court_detection.pt"
    bounce_model_path = "/content/bounce_detection.cbm"

    # 동영상 처리 함수 호출
    success, message = process_video(
        input_video_path=input_video_path,
        output_video_path=output_video_path,
        ball_model=ball_model_path,
        court_model=court_model_path,
        bounce_model=bounce_model_path
    )

    if success:
        st.success("동영상 처리가 완료되었습니다!")
        st.text(message)
    else:
        st.error(f"오류: 동영상 처리 중 문제가 발생했습니다.\n{message}")

    # 동영상 파일 경로 설정
    video_path = '/content/tennis_detection.mp4'  # 처리된 동영상 경로
    output_folder = 'image'  # 저장할 폴더 이름

    # 프레임 별 이미지 함수 호출
    extract_frames_from_video(video_path, output_folder, frame_rate=1)

    # 추출된 이미지 경로 리스트
    image_paths = [
        "/content/image/image0000.jpg",
        "/content/image/image0001.jpg"
    ]

    # 이미지 파일 존재 여부 확인
    missing_files = [path for path in image_paths if not os.path.exists(path)]
    if missing_files:
        st.error(f"다음 이미지 파일이 존재하지 않습니다: {', '.join(missing_files)}")
    else:
        # OpenAI API 호출 및 결과 출력
        st.text("이미지 흐름을 해석 중입니다...")
        interpretation = interpret_images_with_openai(image_paths)
        st.success("해석이 완료되었습니다!")
        st.text(f"해석 결과:\n{interpretation}")

Overwriting app.py


In [ ]:
!ls

app.py	       bounce_detection.cbm  nohup.out	  TennisProject
ball_model.pt  court_detection.pt    sample_data


In [ ]:
!nohup streamlit run app.py --server.port 80 &

nohup: appending output to 'nohup.out'


In [ ]:
!git clone https://github.com/yastrebksv/TennisProject.git
%cd TennisProject

fatal: destination path 'TennisProject' already exists and is not an empty directory.
/content/TennisProject


In [ ]:
import os
os.chdir('/content')

In [ ]:
url = ngrok.connect(addr=80)  # 80번 포트를 노출
url

<NgrokTunnel: "https://ff7a-35-247-124-241.ngrok-free.app" -> "http://localhost:80">

In [ ]:
ngrok.kill()

In [ ]:
#  영상을 가져와서 화면에 출력하는 코드
    # 추출된 첫 번째 프레임 경로
    video_path = os.path.join(output_folder, '/content/tennis_detection.mp4')

    if os.path.exists(video_path):
        # 동영상을 화면에 표시
        st.text("동영상 처리가 완료되었습니다!")
        st.video(video_path)
    else:
        # 동영상 파일이 없을 때 에러 메시지 출력
        st.error("동영상 파일이 존재하지 않습니다. 동영상 생성 과정을 확인하세요.")